# Medium scraper

Now at scrape_medium.py
Inspired by https://github.com/lazuxd/medium-scraping/blob/master/medium_scraping.ipynb 

## Get URLs of articles
Have to first search by month, then if there are more than 10 articles listed, try days

In [6]:
import requests
from bs4 import BeautifulSoup
import pdb
from calendar import Calendar
from datetime import date, timedelta

tag = 'white-supremacy'
url_base = 'https://medium.com/tag/{}/archive/{}/{:02d}'
article_urls = []
c = Calendar()
# for year in range(2010, 2022):
for year in range(2015, 2022):
    for month in range(1, 13):
        month_url = url_base.format(tag, year, month)
        # print(url)

        response = requests.get(month_url, allow_redirects=False)

        if response.status_code != 200:
            print(f"No articles found for {month_url}")
            continue
        soup = BeautifulSoup(response.content)
        articles = soup.find_all("div", class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
        if len(articles) < 10: # Fewer than 10 means don't have to search by day
            print(f'{len(articles)} articles found at {month_url}')
            article_urls += [article.find_all("a")[3]['href'].split('?')[0] for article in articles]
        else:
            calendar_mention = soup.find('p', class_='u-marginBottom40')
            if calendar_mention.text == '': # exactly 10 articles
                print(f'{len(articles)} articles found at {month_url}')
                article_urls += [article.find_all("a")[3]['href'].split('?')[0] for article in articles]
            else: # Try calendar date URLs
                for date in [d for d in c.itermonthdates(year, month) if d.month == month]:
                    day_url = month_url + '/{:02d}'.format(date.day)
                    response = requests.get(day_url, allow_redirects=False)
                    if response.status_code != 200:
                        print(f"No articles found for {day_url}")
                        continue
                    soup = BeautifulSoup(response.content)
                    articles = soup.find_all("div", class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
                    print(f'{len(articles)} articles found at {day_url}')
                    article_urls += [article.find_all("a")[3]['href'].split('?')[0] for article in articles]

No articles found for https://medium.com/tag/white-supremacy/archive/2015/01
1 articles found at https://medium.com/tag/white-supremacy/archive/2015/02
No articles found for https://medium.com/tag/white-supremacy/archive/2015/03
No articles found for https://medium.com/tag/white-supremacy/archive/2015/04
1 articles found at https://medium.com/tag/white-supremacy/archive/2015/05
2 articles found at https://medium.com/tag/white-supremacy/archive/2015/06
5 articles found at https://medium.com/tag/white-supremacy/archive/2015/07
2 articles found at https://medium.com/tag/white-supremacy/archive/2015/08
2 articles found at https://medium.com/tag/white-supremacy/archive/2015/09
10 articles found at https://medium.com/tag/white-supremacy/archive/2015/10
10 articles found at https://medium.com/tag/white-supremacy/archive/2015/11
No articles found for https://medium.com/tag/white-supremacy/archive/2015/11/01
No articles found for https://medium.com/tag/white-supremacy/archive/2015/11/02
No arti

KeyboardInterrupt: 

## Get article text

In [7]:
len(article_urls)

533

In [18]:
import time
import pandas as pd
from tqdm.notebook import tqdm

outlines = []
for url in tqdm(article_urls):
    try:
        response = requests.get(url)
        if response.status_code in [520, 524]:
            sleep_time = 30
            tqdm.write(f'{response.status_code} error, waiting {sleep_time} seconds and then trying again')
            time.sleep(sleep_time)
        elif response.status_code == 200:
            soup = BeautifulSoup(response.content)
            title = ''
            title_search = soup.find('h1', class_='pw-post-title') 
            if title_search is not None:
                title = title_search.text
            paras = [p.text for p in soup.find_all('p', class_='pw-post-body-paragraph')]
            outlines.append({'url': url, 'title': title, 'text': '\n'.join([title] + paras).strip()})
            time.sleep(0.5)
        else: 
            pdb.set_trace()
    except Exception as e:
        tqdm.write(f'{str(e)}\n\t for {url}')
        continue

In [19]:
articles = pd.DataFrame(outlines)
articles

,url,title,text
0,https://medium.com/@hdyerjr/florida-sheriff-te...,Florida Sheriff Tells Drivers to Run over Stre...,Florida Sheriff Tells Drivers to Run over Stre...
1,https://medium.com/@absurdistwords/brunch-an-a...,Brunch: An Allegory of White Supremacy,Brunch: An Allegory of White Supremacy\nFADE I...
2,https://medium.com/news-and-politics/to-my-fri...,"To My Friends Who Have Not Posted, Acknowledge...","To My Friends Who Have Not Posted, Acknowledge..."
3,https://medium.com/@enajlaura/thoughts-on-the-...,Thoughts on the Charleston Shooting (Or: White...,Thoughts on the Charleston Shooting (Or: White...
4,https://medium.com/@Catharticme/teaching-lesso...,Teaching Lessons of White Supremacy in Prime-T...,Teaching Lessons of White Supremacy in Prime-T...
...,...,...,...
134,https://bullshit.ist/running-on-high-velocity-...,Running on “high-velocity bitterness” on my “o...,Running on “high-velocity bitterness” on my “o...
135,https://medium.com/@abelateiner/the-gift-of-wh...,The Gift of White Burnout,The Gift of White Burnout\nBurnout saved my li...
136,https://medium.com/@YcfMan/here-is-my-solution...,,Here is my solution to staying alive: get a re...
137,https://medium.com/@joselromero/the-one-absolu...,The One Absolute Reason Why Racism Is Bullshit!,The One Absolute Reason Why Racism Is Bullshit...


In [20]:
pd.set_option('display.max_colwidth', None)
articles.head()

url  \
0                                             https://medium.com/@hdyerjr/florida-sheriff-tells-drivers-to-run-over-street-protesters-bebdddde9d82   
1                                                            https://medium.com/@absurdistwords/brunch-an-allegory-of-white-supremacy-1e5b9344dd5e   
2        https://medium.com/news-and-politics/to-my-friends-who-have-not-posted-acknowledged-shared-or-commented-about-the-tragedy-in-a00798464010   
3                https://medium.com/@enajlaura/thoughts-on-the-charleston-shooting-or-white-supremacy-continues-to-be-awful-pervasive-8ffba481a8f2   
4  https://medium.com/@Catharticme/teaching-lessons-of-white-supremacy-in-prime-time-blackrifice-in-the-post-apocalyptic-world-of-the-53213182050f   

                                                                                                              title  \
0                                                       Florida Sheriff Tells Drivers to Run over Street Protesters   
1                                                                            Brunch: An Allegory of White Supremacy   
2              To My Friends Who Have Not Posted, Acknowledged, Shared or Commented About the Tragedy in Charleston   
3                        Thoughts on the Charleston Shooting (Or: White Supremacy Continues to Be Awful, Pervasive)   
4  Teaching Lessons of White Supremacy in Prime-Time: Blackrifice in the Post-Apocalyptic World of the CW’s The 100   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               